In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import fpfs
import fitsio
import impt
import jax.numpy as jnp

In [2]:
test_fname = os.path.join(
    impt.fpfs.__data_dir__,
    "fpfs-cut32-0000-g1-0000.fits",
)

# impt.fpfs
cat = impt.fpfs.read_catalog(test_fname)
# FPFS
data = fitsio.read(test_fname)
cov_mat2 = impt.fpfs.utils.fpfscov_to_imptcov(data)

2023/01/11 06:50:25 ---  Remote TPU is not linked into jax; skipping remote TPU.
2023/01/11 06:50:25 ---  Unable to initialize backend 'tpu_driver': Could not initialize backend 'tpu_driver'
2023/01/11 06:50:25 ---  Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2023/01/11 06:50:25 ---  Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2023/01/11 06:50:25 ---  Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
2023/01/11 06:50:25 ---  Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.
2023/01/11 06:50:25 ---  No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
params = impt.fpfs.FpfsParams(
    Const = 2.0,
    lower_m00=0.5,
    sigma_m00=0.5,
    lower_r2=0.12,
    sigma_r2=0.3,
    sigma_v=0.3,
)
e1_impt = impt.fpfs.FpfsE1(params)
e2_impt = impt.fpfs.FpfsE2(params)
w_det = impt.fpfs.FpfsWeightDetect(params)
w_sel = impt.fpfs.FpfsWeightSelect(params)
we1 = e1_impt * w_det * w_sel
we2 = e2_impt * w_det * w_sel

dwe1_dg1 = impt.RespG1(we1)
dwe2_dg2 = impt.RespG2(we2)

In [6]:
de1_dg1 = impt.RespG1(e1_impt)

In [7]:
ell_fpfs = fpfs.catalog.fpfs_m2e(data, const=params.Const, noirev=False)
fs = fpfs.catalog.summary_stats(data, ell_fpfs, use_sig=False, ratio=1.0)
selnm = np.array(["detect2", "M00", "R2"])
fs = impt.fpfs.test_utils.initialize_FPFS(fs, selnm, params)

In [8]:
ell_fpfs2 = fpfs.catalog.fpfs_m2e(data, const=params.Const, noirev=True)
fs2 = fpfs.catalog.summary_stats(data, ell_fpfs2, use_sig=False, ratio=1.0)
fs2 = impt.fpfs.test_utils.initialize_FPFS(fs2, selnm, params)

In [11]:
jnp.sum(we1.evaluate(cat)) + jnp.sum(we2.evaluate(cat))

Array(-0.0229117, dtype=float64)

In [19]:
bnoise = impt.BiasNoise(we1, cov_mat2)
jnp.sum(bnoise.evaluate(cat))

Array(0.17612553, dtype=float64)

In [17]:
fs2.sumE1 - fs.sumE1 + fs2.corE1

-0.003958492561487458

In [10]:
fs2.sumE1 + fs2.sumE2

-0.022973782551999403

In [104]:
fs2.sumR1 + fs2.sumR2 + fs2.corR1 + fs2.corR2

0.022308760428894524

In [105]:
fs2.sumE1 + fs2.sumE2 + fs2.corE1 + fs2.corE2

-0.009794882138155086

In [106]:
fs2.corE1 + fs2.corE2

-1.715909172982086e-05

In [15]:
np.sum(ell_fpfs['fpfs_e1']-ell_fpfs2['fpfs_e1'])

-0.0050717731940334194

In [13]:
e1_final = impt.fpfs.FpfsWeightE1(params)
bnoise = impt.BiasNoise(e1_final, cov_mat2)
jnp.sum(bnoise.evaluate(cat))

Array(0.17612553, dtype=float64)

In [14]:
bnoise = impt.BiasNoise(we1, cov_mat2)
jnp.sum(bnoise.evaluate(cat))

Array(0.17612553, dtype=float64)

In [109]:
e1_final = impt.fpfs.FpfsWeightE1(params)
bnoise = impt.BiasNoise(e1_final, cov_mat2)
jnp.sum(bnoise.evaluate(cat))

Array(1.67618787e-05, dtype=float64)

In [81]:
e1_final = impt.fpfs.FpfsWeightE1(params)
res1 = impt.RespG1(e1_final)
bnoise = impt.BiasNoise(res1, cov_mat2)
jnp.sum(bnoise.evaluate(cat))

Array(-6.07610356, dtype=float64)

In [82]:
e1_final = impt.fpfs.FpfsWeightE1(params)
bnoise = impt.BiasNoise(e1_final, cov_mat2)
jnp.sum(bnoise.evaluate(cat))

Array(0.17612553, dtype=float64)

In [83]:
bnoise = impt.BiasNoise(we1, cov_mat2)
jnp.sum(bnoise.evaluate(cat))

Array(0.17612553, dtype=float64)

In [84]:
jnp.sum(bnoise.evaluate(cat))

Array(0.17612553, dtype=float64)

In [85]:
fs.sumR1 + fs.sumR2 + fs.corR1 + fs.corR2

2.4798659683593103

In [86]:
import fitsio
cov_fname = os.path.join(impt.fpfs.__data_dir__, "modes_cov_mat_paper3_045.fits")
cov_mat = fitsio.read(cov_fname)

In [87]:
bnoise=impt.BiasNoise(dwe1_dg1, jnp.array(cov_mat))
jnp.sum(bnoise.evaluate(cat))

Array(-5.86349852, dtype=float64)

In [88]:
bnoise=impt.BiasNoise(dwe2_dg2, jnp.array(cov_mat))
jnp.sum(bnoise.evaluate(cat))

Array(-3.95003624, dtype=float64)

In [90]:
cov_mat2 = impt.fpfs.utils.fpfscov_to_imptcov(data)

In [92]:
bnoise2=impt.BiasNoise(de1_dg1, jnp.array(cov_mat))
jnp.sum(bnoise2.evaluate(cat))

Array(0.19156662, dtype=float64)

In [93]:
bnoise2=impt.BiasNoise(de1_dg1, jnp.array(cov_mat2))
jnp.sum(bnoise2.evaluate(cat))

Array(0.19156663, dtype=float64)

In [94]:
params

FpfsParams(Const=2.0, lower_m00=3.5, sigma_m00=0.5, lower_r2=0.12, upper_r2=2.0, sigma_r2=0.3, lower_v=0.005, sigma_v=0.3)